In [14]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [37]:
import pandas as pd

df = pd.read_csv('data/rated_2014-01_initial_cleaning.csv')
df = df[df['event'] == 'classical']
df = df[df.columns[1:]]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 246495 entries, 2 to 697466
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   white_elo     246495 non-null  float64
 1   black_elo     246495 non-null  float64
 2   time_control  246495 non-null  object 
 3   result        246495 non-null  object 
 4   termination   246495 non-null  object 
 5   moves         246495 non-null  object 
dtypes: float64(2), object(4)
memory usage: 13.2+ MB


In [41]:
df.to_csv('data/classical-2024-01-init.csv')

In [16]:
import fasttext
embeddings_model = fasttext.load_model('models/chess2vec-3ws-32dim.bin') 

In [17]:
from utils.data import ChessDataset
from torch.utils.data import DataLoader
dataset = ChessDataset(df, embeddings_model=embeddings_model, device=device)

In [48]:
from torch.utils.data import random_split
from utils.data import collate_data

dataset, _ = random_split(dataset, [0.1, 0.9])

train_dataset, valid_dataset = random_split(dataset, [0.8, 0.2])
train_loader = DataLoader(train_dataset, batch_size=128, collate_fn=collate_data, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=128, collate_fn=collate_data, shuffle=False)

In [49]:
train_dataset.__len__()

198

In [53]:
from torch.optim import Adam
from torch.nn import MSELoss
import utils.models as models
import importlib
importlib.reload(models)
model = models.ELOPredictor()
optimizer = Adam(model.parameters())
loss = MSELoss()

model = torch.compile(model.to(device))

In [ ]:
from torch.utils.tensorboard.writer import SummaryWriter

SUMMARY_COMMENT = "exper"

writer = SummaryWriter('./logs/', comment=SUMMARY_COMMENT)

epochs = 3
for epoch in range(epochs):
    print(f'Epoch: {epoch+1}')
    train = models.train(model, train_loader, optimizer, loss, device)
    valid = models.validate(model, valid_loader, loss, device)
    
    observs = dict(train)
    observs.update(valid)
    
    writer.add_scalars("exper", observs, epoch)
writer.flush()

Epoch: 1


training:   0%|          | 0/2 [00:00<?, ?it/s]

Train - Loss: 1.8694


validation:   0%|          | 0/1 [00:00<?, ?it/s]

Validation - Loss: 0.8464	r2_score: tensor([ 0.0475, -0.0178])	mse_result: tensor([0.8310, 0.8619])
Epoch: 2


...

Validation - Loss: 0.8497	r2_score: tensor([ 0.0523, -0.0305])	mse_result: tensor([0.8268, 0.8726])
Epoch: 3


training:   0%|          | 0/2 [00:00<?, ?it/s]

Train - Loss: 1.7845


validation:   0%|          | 0/1 [00:00<?, ?it/s]

Validation - Loss: 0.8441	r2_score: tensor([ 0.0606, -0.0258])	mse_result: tensor([0.8195, 0.8687])


In [61]:
import os
os.environ['TENSORBOARD_BINARY'] = '/home/zamog/.cache/pypoetry/virtualenvs/non-package-mode-uH4eZcH1-py3.11/bin/tensorboard'

In [63]:
%load_ext tensorboard
%tensorboard --logdir ./logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 175284), started 0:00:28 ago. (Use '!kill 175284' to kill it.)